# Proof-of-concept demo of Merkle Octree

This notebook demonstrates the use of an [octree](https://en.wikipedia.org/wiki/Octree) data structure whose nodes also form a [Merkle Tree](https://en.wikipedia.org/wiki/Merkle_tree).
The Merkle octree can be used to quickly detect and determine map changes to enable efficient sharing of map updates in a distributed mapping scenario.

In [1]:
from copy import deepcopy
from merkleoctree import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create a single node tree

tree_A = MerkleOctree(min_bounds=[-10,-10,-10], max_bounds=[10,10,10], max_depth=8)

tree_A.insert(DataPoint(pos=[1,1,1], data=NumpyDataType([1.,1.,1.]))) # Note: data could be any binary-encoded data (occpancy, semantic classification belief)

print('=== Tree A: Before changes ===')
hashes_A = tree_A.hash_tree

for node in hashes_A:
    print(node)

=== Tree A: Before changes ===
(7, 0, 0, b'TWaD\xb1\x06\x1a\xbdN0x\xd7\xa2\xfbj\xf1?\xe40^')
(6, 0, 7, b'"\xb3\x8c\x19\xb7\x15}\xb5\xdf\xc1\xa1f.*\x06\x8e,\xab\xf7\xae')
(5, 7, 0, b'\xe4\x8a\xac\x88\x05\x9e\x1bCQ\xd9\xcc:\x19\xcbd^\xf6@\xd2\xee')
(4, 0, 0, b's\xe3\x98;\xd5\xc1\x11\xca\x84z\xe1\xff\xfe\x89]f\x8dp\x0b\xce')
(3, 0, 0, b'\xce\x89\x81;\xef\xfa\x05\xc7\xf7\x10,&\x11\xb1\xaa\x88\xc0*\xbay')
(2, 0, 7, b'\x1f\x90\x85\x02\xd0\xcbA\xe7\x9b&\x93+z\xa2\x811_\x04\xbe ')
(1, 7, 7, b'\xe8w \xb7>\xf6c\x87\xd8\xab\x0f\xea2\xe7\xceN\xfa\xa6t\xaa')
(0, 7, 0, b'\x8a\xaa\xb8\xea=\xc5\xe1\x12\x7f,\xda\xec1*j\xc5\xfa\xb8\x0e&')


In [3]:
# We modify the existing data point by inserting different data to the same location.

# Note that all the hashes change, indicating at each level that a child value has changed.
# A fast change detection for the whole tree can be done by comparing only the top hash.

tree_B = deepcopy(tree_A)
tree_B.insert(DataPoint([1,1,1], NumpyDataType([2.,2.,2.])))

print('=== Tree B: After 1st change ===')

hashes_B = tree_B.hash_tree

for node in hashes_B:
    print(node)

=== Tree B: After 1st change ===
(7, 0, 0, b'\x03\xb61\xf6M%\x01>:\xe0<c=B-1&\x89\x96W')
(6, 0, 7, b"\r\xbb-\xf7\x84^\xbf\xf3\x0e\xa6\x0c\xdfr'\xb4(\x13I\xdc\x0f")
(5, 7, 0, b'S\xe4\xae\x06u\xec\x98\x82\xcb\x1b\t%5\xcc\xd0U\xc7\x8f\x990')
(4, 0, 0, b'\xba\xfcF\x07\xbe\xcdM\xc6\x10\x9f\xe00\x07\x95\x11\x8f.\x85\x98.')
(3, 0, 0, b'v\x9f:\xd0\xc4\x8c<oYT\xdc\xbf\x8e\xc4\xc1\x0f\xb6\x07\x98p')
(2, 0, 7, b'\x8b\x14\x0b\xbd\xec\xecC\xe9\x9a7\xc4\x87\x14\xb2Uu\xa5\x05\xb4I')
(1, 7, 7, b'\xaa\x909\x9eKt\x18\xe0\xd0\x96\xc1\xcd\xce\x8b7\xb8\x87\x83\x1c\x87')
(0, 7, 0, b'\x95L\xd9\x7f$\x8d\xb03b\xda\x16<`{=\xeb&vx^')


In [4]:
# We insert a new data point in a different location.

# Note that more hashes are shown, indicating the tree expanding to accomodate the new point.
# The space-partitioning tree structure is "right-sized" for the sparse data, and does not allocate unneeded empty voxels.

# Note that the top hash changes indicating a change in the tree.
# However hashes in the branch starting with (6, 0, 7, ...) remain the same, indicating no change in that branch, allowing 
# it to be skipped in a map update.

tree_C = deepcopy(tree_B)
tree_C.insert(DataPoint([-1,1,-1], NumpyDataType([1.,2.,3.])))

print('=== Tree C: After 2nd change ===')

hashes_C = tree_C.hash_tree

for node in hashes_C:
    print(node)

=== Tree C: After 2nd change ===
(7, 0, 0, b'\xb4{\xca\xe1\x91\xeb$\xb7%zS\xb5\xa5F\xea!u>Lj')
(6, 0, 7, b"\r\xbb-\xf7\x84^\xbf\xf3\x0e\xa6\x0c\xdfr'\xb4(\x13I\xdc\x0f")
(5, 7, 0, b'S\xe4\xae\x06u\xec\x98\x82\xcb\x1b\t%5\xcc\xd0U\xc7\x8f\x990')
(4, 0, 0, b'\xba\xfcF\x07\xbe\xcdM\xc6\x10\x9f\xe00\x07\x95\x11\x8f.\x85\x98.')
(3, 0, 0, b'v\x9f:\xd0\xc4\x8c<oYT\xdc\xbf\x8e\xc4\xc1\x0f\xb6\x07\x98p')
(2, 0, 7, b'\x8b\x14\x0b\xbd\xec\xecC\xe9\x9a7\xc4\x87\x14\xb2Uu\xa5\x05\xb4I')
(1, 7, 7, b'\xaa\x909\x9eKt\x18\xe0\xd0\x96\xc1\xcd\xce\x8b7\xb8\x87\x83\x1c\x87')
(0, 7, 0, b'\x95L\xd9\x7f$\x8d\xb03b\xda\x16<`{=\xeb&vx^')
(6, 0, 2, b'V\x89\xc4\xebh\xd7. \x1d\xdb\x8a\x95azA\xe4\xe9\x95g\xbc')
(5, 2, 5, b'X\xb5\x89\x83L\xb6\xe1}\xd1\x96R\xa3\xf5\xda\xeb\x08>\x10\xae\xb6')
(4, 5, 5, b'of\xcd\xb1\xfc\xb8\xe4\x7f\x86TsN\x8f>\x05\xf8\x1c\xce \xf8')
(3, 5, 5, b'\xdf\xa0\xb8\x98\xe3\x1a\x1c\xc5\xdd\x13E\xd0\xe3\x04\x8c\xc7M\xcet\xa1')
(2, 5, 2, b"\x1eS\xb6\x8d\xf1\xe7\x1d\xb2'k. \x12c\xd6|\x8c\xebP}")


In [5]:
# We demonstrate how the hash tree might be shared independent of the underlying map data.

# Encode the hash tree as a list (can be further packed into binary with compression)
hashes_C = list(tree_C.hash_tree)
print('=== Tree C Hashes as list ===')
for node in hashes_C:
    print(node)

# Recreate a hash-only Merkle tree from the encoded hash list
# Note that the reconstructed hashes are the same as the original.
hash_tree_C = HashTree.create_from_encoded(hashes_C)

print('=== Tree C Hash-only Reconstruction ===')
for node in hash_tree_C:
    print(node)

=== Tree C Hashes as list ===
(7, 0, 0, b'\xb4{\xca\xe1\x91\xeb$\xb7%zS\xb5\xa5F\xea!u>Lj')
(6, 0, 7, b"\r\xbb-\xf7\x84^\xbf\xf3\x0e\xa6\x0c\xdfr'\xb4(\x13I\xdc\x0f")
(5, 7, 0, b'S\xe4\xae\x06u\xec\x98\x82\xcb\x1b\t%5\xcc\xd0U\xc7\x8f\x990')
(4, 0, 0, b'\xba\xfcF\x07\xbe\xcdM\xc6\x10\x9f\xe00\x07\x95\x11\x8f.\x85\x98.')
(3, 0, 0, b'v\x9f:\xd0\xc4\x8c<oYT\xdc\xbf\x8e\xc4\xc1\x0f\xb6\x07\x98p')
(2, 0, 7, b'\x8b\x14\x0b\xbd\xec\xecC\xe9\x9a7\xc4\x87\x14\xb2Uu\xa5\x05\xb4I')
(1, 7, 7, b'\xaa\x909\x9eKt\x18\xe0\xd0\x96\xc1\xcd\xce\x8b7\xb8\x87\x83\x1c\x87')
(0, 7, 0, b'\x95L\xd9\x7f$\x8d\xb03b\xda\x16<`{=\xeb&vx^')
(6, 0, 2, b'V\x89\xc4\xebh\xd7. \x1d\xdb\x8a\x95azA\xe4\xe9\x95g\xbc')
(5, 2, 5, b'X\xb5\x89\x83L\xb6\xe1}\xd1\x96R\xa3\xf5\xda\xeb\x08>\x10\xae\xb6')
(4, 5, 5, b'of\xcd\xb1\xfc\xb8\xe4\x7f\x86TsN\x8f>\x05\xf8\x1c\xce \xf8')
(3, 5, 5, b'\xdf\xa0\xb8\x98\xe3\x1a\x1c\xc5\xdd\x13E\xd0\xe3\x04\x8c\xc7M\xcet\xa1')
(2, 5, 2, b"\x1eS\xb6\x8d\xf1\xe7\x1d\xb2'k. \x12c\xd6|\x8c\xebP}")
(1,

In [6]:
# We demonstrate how quick difference determination can be done.

# We also reconstruct hash-only Merkle trees for Trees A and B.
hashes_A = list(tree_A.hash_tree)
hash_tree_A = HashTree.create_from_encoded(hashes_A)

hashes_B = list(tree_B.hash_tree)
hash_tree_B = HashTree.create_from_encoded(hashes_B)

# Compare A against A. No changes.
print('=== A against A ===')
print(compare_hash_trees(hash_tree_A, hash_tree_A))

# Compare B against A. Need to merge 1 leaf node.
print('=== B against A ===')
print(compare_hash_trees(hash_tree_A, hash_tree_B))

# Compare C against B. Need to add a branch.
print('=== C against B ===')
print(compare_hash_trees(hash_tree_B, hash_tree_C))

# Compare C against A. Need to merge 1 leaf node and add a branch.
print('=== C against A ===')
print(compare_hash_trees(hash_tree_A, hash_tree_C))


=== A against A ===
[]
=== B against A ===
[([7, 0, 0, 0, 7, 7, 0], <ChangeStatus.MERGE_BOTH: 3>)]
=== C against B ===
[([2], <ChangeStatus.ADD_TO_LOCAL: 1>)]
=== C against A ===
[([2], <ChangeStatus.ADD_TO_LOCAL: 1>), ([7, 0, 0, 0, 7, 7, 0], <ChangeStatus.MERGE_BOTH: 3>)]


In [7]:
# We demonstrate how patch changes can be extracted from difference trees.
# These patches are the minimal changes required.

changes = extract_changes(tree_C, compare_hash_trees(tree_C, tree_A))
for c in changes:
    print(c)

# Note: The change DataPoints have a different position from the original
#       as the octree voxelizes the inserted points.

<DataPoint: pos=[-1.015625  1.015625 -1.015625], data=<NumpyDataType: data=[4.9e-324 9.9e-324 1.5e-323]>>
<DataPoint: pos=[1.015625 1.015625 1.015625], data=<NumpyDataType: data=[1.e-323 1.e-323 1.e-323]>>
